<a href="https://colab.research.google.com/github/larasauser/master/blob/main/cloud_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio

In [7]:
from google.colab import drive
drive.flush_and_unmount()  # démonte si déjà monté
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.


ValueError: Mountpoint must not already contain files

In [5]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np

# --- Chemins ---
cloud_geom_path = "/content/drive/MyDrive/cloud_geom/cloud_geometry_2023-11-24.geojson"
input_folder = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed"
output_folder = "/content/drive/MyDrive/NDVI_grancy_Landsat8_10/imputed/holes_real"

# Créer le dossier de sortie si nécessaire
os.makedirs(output_folder, exist_ok=True)

# --- Charger le GeoJSON ---
cloud_gdf = gpd.read_file(cloud_geom_path)
cloud_geom = [feature["geometry"] for feature in cloud_gdf.__geo_interface__["features"]]

# --- Boucle sur toutes les images ---
for fname in os.listdir(input_folder):
    if fname.endswith(".tif"):
        input_path = os.path.join(input_folder, fname)
        output_path = os.path.join(output_folder, fname)

        with rasterio.open(input_path) as src:
            data = src.read(1).astype(np.float32)
            profile = src.profile

            # Appliquer le masque: les pixels à l'intérieur des polygones deviennent NaN
            out_image, out_transform = mask(src, cloud_geom, invert=False, filled=True, nodata=np.nan)
            out_image = out_image[0]  # Extraire la bande unique

            # Copier les pixels hors nuage et mettre NaN dans le nuage
            # Ici, rasterio.mask already sets pixels outside mask to NaN if invert=False
            # Donc out_image est prêt à être sauvegardé

            profile.update(dtype=rasterio.float32, nodata=np.nan)

            with rasterio.open(output_path, "w", **profile) as dst:
                dst.write(out_image, 1)

        print(f"Created cloud hole image: {output_path}")


DataSourceError: /content/drive/MyDrive/cloud_geom/cloud_geometry_2023-11-24.geojson: No such file or directory

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import os

# Dossier des images trouées
holes_folder = "/content/drive/MyDrive/NDVI_herens_Landsat8_10/imputed/holes_real"
all_files = [f for f in os.listdir(holes_folder) if f.endswith(".tif")]

# Sélectionner 3 images aléatoires
sample_files = random.sample(all_files, min(3, len(all_files)))

fig, axes = plt.subplots(1, len(sample_files), figsize=(5*len(sample_files),5))

for ax, fname in zip(axes, sample_files):
    path = os.path.join(holes_folder, fname)
    with rasterio.open(path) as src:
        img = src.read(1)

    # Remplacer NaN par rouge pour visualisation
    display_img = np.copy(img)
    mask_nan = np.isnan(display_img)
    display_img[mask_nan] = -1.5  # valeur hors plage pour rouge

    cmap = plt.cm.gray
    cmap.set_bad(color='red')  # si tu veux, on peut aussi utiliser NaN directement

    im = ax.imshow(display_img, cmap='gray', vmin=-1, vmax=1)
    ax.set_title(fname, fontsize=10)
    ax.axis('off')

plt.colorbar(im, ax=axes, fraction=0.02, pad=0.02, label="NDVI")
plt.show()
